In [1]:
# import os
# from huggingface_hub import InferenceClient
# from datetime import datetime

# client = InferenceClient(
#     api_key=os.environ["HF_TOKEN"],
# )

# # System prompt for pseudo code generation
# SYSTEM_PROMPT = """You are an expert pseudo code generator. Your task is to:
# 1. Analyze control narrative documents
# 2. Extract key processes and workflows
# 3. Convert them into clear, well-structured pseudo code
# 4. Use proper indentation and logical flow
# 5. No actual code syntax - only pseudo code notation
# 6. No explanations or additional text - only the pseudo code
# 7. And no extras, just pseudo code based on the given information."""

# user_input = "Write pseudo code based on this control narrative document:"

# # Convert bytes to string
# full_text_str = "\n".join([text.decode("utf-8") if isinstance(text, bytes) else text for text in full_text])

# # print("\n" + "="*80)
# # print("PSEUDO CODE GENERATION - HuggingFace API")
# # print("="*80)
# # print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
# # print(f"Model: openai/gpt-oss-20b")
# # print("="*80 + "\n")

# completion = client.chat.completions.create(
#     model="openai/gpt-oss-20b",
#     messages=[
#         {
#             "role": "user",
#             "content": SYSTEM_PROMPT + "\n\n" + user_input + "\n\n" + full_text_str
#         }
#     ],
#     max_tokens=2048,
#     temperature=0.7,
# )

# response_text = completion.choices[0].message.content

# print(response_text)


In [2]:
import pymupdf

doc = pymupdf.open("document.pdf")

In [3]:
text_blocks = []
image_blocks = []

In [4]:
def extract_pdf_content(pdf_path):
    doc = pymupdf.open(pdf_path)

    text_collection = []
    image_collection = []

    for page_num, page in enumerate(doc):
        page_dict = page.get_text("dict")

        for block_id, block in enumerate(page_dict["blocks"]):
            if block["type"] == 0:
                text = " ".join(
                    span["text"]
                    for line in block["lines"]
                    for span in line["spans"]
                ).strip()

                if text:
                    text_collection.append({
                        "id": f"text_p{page_num}_b{block_id}",
                        "content": text,
                        "metadata": {
                            "page": page_num,
                            "bbox": block["bbox"],
                            "modality": "text"
                        }
                    })
        
        for img_id, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base = doc.extract_image(xref)

            image_collection.append({
                "id": f"image_p{page_num}_{img_id}",
                "image_bytes": base["image"],
                "format": base["ext"],
                "metadata": {
                    "page": page_num,
                    "xref": xref,
                    "modality": "image"
                }
            })
    return text_collection, image_collection

In [5]:
texts, images = extract_pdf_content("document.pdf")

print(len(texts), "text chunks")
print(len(images), "images")


845 text chunks
46 images


In [6]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="devstral-2512",
    temperature=0.2,
)

response = llm.invoke("You are just coding bot, so dont give any other content just asked code. write code for simple neural network in pytorch")
print(response.content)


```python
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Example usage
input_size = 10
hidden_size = 5
output_size = 1
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop (example)
for epoch in range(100):
    # Generate random data
    inputs = torch.randn(32, input_size)
    targets = torch.randn(32, output_size)

    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.

In [1]:
from langchain_mistralai import ChatMistralAI
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFLoader, UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import os


2025-12-26 23:19:32.322278: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-26 23:19:32.322902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-26 23:19:32.380259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-26 23:19:34.100041: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
PDF_PATH = "document.pdf"

In [3]:
try:
    loader = PyPDFLoader(PDF_PATH)
    docs = loader.load()
    if len("".join([d.page_content for d in docs]).strip()) == 0:
        raise Exception("No text layer")
except:
    loader = UnstructuredPDFLoader(PDF_PATH, mode="elements")
    docs = loader.load()

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=400
)
chunks = splitter.split_documents(docs)


In [5]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 12})


/tmp/ipykernel_34253/3136526489.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [6]:
coverage_prompt = ChatPromptTemplate.from_template("""
You are a CONTROL LOGIC REQUIREMENT EXTRACTOR.

From the context, extract ONLY actionable engineering control logic.
Do NOT write pseudocode here.
Do NOT summarize narrative.
Extract structured, bullet logic.

YOU MUST extract:

1. Startup requirements
2. Feed weight / feed flow / feed logic
3. Alarm strategy rules and compliance expectations
4. Shutdown behavior logic
5. Abort behavior logic
6. Surge vessel behavior and divert logic
7. Depth filtration handling and routing logic
8. State machine integrity and transition rules
9. Interlocks, permissives, safety behavior
10. Any timing, overrides, or dependency constraints

Write as structured engineering logic statements.

<context>
{context}
</context>
""")

In [7]:
def join_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [17]:
llm = ChatMistralAI(
    model="devstral-2512",
    temperature=0.0,
    max_tokens=4096*2
)

In [18]:
coverage_chain = (
    {
        "context": retriever | join_docs
    }
    | coverage_prompt
    | llm
)

In [23]:
GENERIC_RAG_QUERY = (
    "extract complete control / system logic requirements including "
    "startup conditions, operating logic, decision rules, alarms, interlocks, "
    "shutdown behavior, abort handling, state transitions, subsystem behavior, "
    "and compliance expectations"
)

coverage_result = coverage_chain.invoke(GENERIC_RAG_QUERY).content


In [24]:
pseudocode_prompt = ChatPromptTemplate.from_template("""
{rules}

<context>
{context}
</context>

TASK: Generate FULL and COMPLETE pseudocode including ALL required logic.
""")

RULE_BLOCK = """
You are a STRICT PSEUDOCODE GENERATOR.

You MUST FOLLOW THESE RULES EXACTLY:

GENERAL RULES
1. Output ONLY executable logic and control flow.
2. Do NOT generate narrative, descriptions, titles, metadata, revision history, dates, headers, or table of contents.
3. Do NOT output quotes or literal printed document text.
4. Ignore any non-executable content.
5. Infer missing logic when implied.

STRUCTURE RULES
6. ALL CONTROL KEYWORDS MUST BE CAPITAL:
   BEGIN, END, FUNCTION, RETURN, IF, ELSE, ELSE IF, FOR, WHILE
7. Use meaningful variable names aligned to document terminology.
8. Maintain clear indentation and hierarchy.
9. Only pseudocode. No commentary.

MANDATORY COVERAGE
10. MUST include:
    Startup logic
    Feed logic
    Alarm strategy
    Shutdown logic
    Abort logic
    Surge vessel logic
    Depth filtration logic
11. Merge multi-section logic into continuous execution flow.
12. Do not omit any relevant logic.

QUALITY
13. Deterministic and runnable in concept.
14. Valid state model.
15. No vague placeholders.
"""

In [25]:
final_chain = (
    {
        "rules": lambda _: RULE_BLOCK,
        "context": lambda _: coverage_result
    }
    | pseudocode_prompt
    | llm
)


In [26]:
response = final_chain.invoke({})
print(response.content)

```
FUNCTION Startup()
    BEGIN
        WAIT FOR operator confirmation: filter installed and flushed
        WAIT FOR operator confirmation: ready to tare pressure transducers (PT01–PT08)
        WAIT FOR operator confirmation: ready to tare flow transducers (FIT01–FIT07)
        WAIT FOR operator confirmation: ready to tare Feed SUSV UV (AT03)

        WAIT FOR Feed SUSV weight > minimum weight limit

        IF pH control enabled THEN
            SET Feed SUSV pH EM command TO pH control
            ENABLE Feed SUSV pH High/Low alarms
        END IF

        IF conductivity control enabled THEN
            SET Feed SUSV conductivity EM command TO conductivity control
            ENABLE Feed SUSV conductivity High/Low alarms
        END IF

        ENABLE Feed SUSV weight High/Low alarms

        IF pH control enabled OR conductivity control enabled THEN
            WAIT FOR pH AND conductivity values in range
        END IF
    END
END FUNCTION

FUNCTION FeedLogic()
    BEGIN
      